In [ ]:
import keras
import keras.backend as K
import tensorflow
from matplotlib import pyplot as plt
import numpy as np
from keras.layers import Input, Conv2D, MaxPooling3D, UpSampling3D, Activation, Dense, Reshape, Flatten,Concatenate
from keras.layers import Conv2DTranspose, ConvLSTM2D, TimeDistributed, Lambda, Bidirectional, Add
from keras_layer_normalization import LayerNormalization
from keras.utils.generic_utils import get_custom_objects
from keras.models import Model
from tensorflow.keras.optimizers import Adam
import h5py
import os
import warnings
warnings.filterwarnings('ignore')

from generate_data import *
from ALADDIn_model import *
from lambda_layers import *
from process_output import *

# Initialize input parameters

In [ ]:
DATASET_PATH =""
Time_seq_sz = 9
N_IFG = 26
Overlapping_epochs = 5
input_img = Input(shape = (N_IFG, 256, 256, 1))  #input interferograms
input_img_TS = Input(shape = (Overlapping_epochs, 256, 256, 1)) #input overlapping epoch time-series
input_ts_ep = Input(shape = (Overlapping_epochs, 256, 256, 1), name = 'image_ts') #input overlapping epoch time-series
input_img_ifgm = Input(shape = (N_IFG, 256, 256, 1), name = 'image_ifgm') #input interferograms
get_custom_objects().update({'gelu': Activation(gelu)})

# Define network architecture and load pre-trained model files

In [ ]:
ALADDIn = Model(input_img, create_model(input_img)) # load network architecture of ALADDIn

ALADDIn.compile(optimizer=tensorflow.keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, amsgrad=False), loss='mean_squared_error')
file_name_model = 'Trained_models/ALADDIn.hdf5'  # pre-trained model file name
layer_output = ALADDIn.get_layer('TS').output     # get output for epoch time-series
ALADDIn_intermediate_model = Model(inputs=ALADDIn.input,outputs=layer_output)
ALADDIn_intermediate_model.load_weights(file_name_model) # load pre-trained ALADDIn mode
print(ALADDIn_intermediate_model.summary())

In [ ]:
### break layers from ALADDIN's decoder
ALADDIn_break_decoder = Model(inputs=ALADDIn.input, outputs=ALADDIn_intermediate_model.layers[43].output)
print(ALADDIn_break_decoder.summary())

In [ ]:
TS_feat = ALADDIn_break_decoder(input_img_ifgm)  ### features for overlapping sequence

### separate features of first 5 and last 4 epochs
First_5_Ep = Lambda(merge_TS, name ='merge_TS')(TS_feat)
Last_4_Ep = Lambda(merge_TS_end, name ='merge_TS_end')(TS_feat)

### define model (encoder) for overlapping epochs input of previous sequence
x1_ts = TimeDistributed(Conv2D(32, (5, 5), strides=4, activation="tanh", padding="same"), batch_input_shape=(None, 5, 256, 256, 1))(input_img_TS)
x1_ts = LayerNormalization()(x1_ts)
x1_ts = TimeDistributed(Conv2D(64, (5, 5), strides=2, activation="tanh", padding="same"))(x1_ts)
x1_ts = LayerNormalization()(x1_ts)
    
x1_ts = ConvLSTM2D(64, (3, 3), activation='tanh', padding="same", return_sequences=True)(x1_ts)
x1_ts = LayerNormalization()(x1_ts)

t_half_2_t_half = Model(inputs = input_img_TS, outputs = x1_ts, name = 'TS_to_TS')
print(t_half_2_t_half.summary())

In [ ]:
### Input overlapping epochs from previous sequence
half_ts_feat = t_half_2_t_half(input_ts_ep)

### Add features of overlapping epochs 
filtered_f_5_ep = Add()([First_5_Ep, half_ts_feat])

### Join back the filtered epochs and last 4 epochs & pass it on to the decoder
m_TS = Concatenate(axis=1)([filtered_f_5_ep, Last_4_Ep])

x15 = TimeDistributed(Conv2DTranspose(128, (3, 3), activation="gelu",padding="same", kernel_initializer='glorot_normal'), batch_input_shape=(None, 9, 256, 256, 1))(m_TS)
x15 = LayerNormalization()(x15)
c7 = Concatenate(axis=-1)([x15, m_TS])
x16 = TimeDistributed(Conv2DTranspose(128, (3, 3), strides =2, activation="gelu",padding="same", kernel_initializer='glorot_normal'))(c7)
x16 = LayerNormalization()(x16)
x17 = TimeDistributed(Conv2DTranspose(64, (3, 3), activation="gelu", padding="same", kernel_initializer='glorot_normal'))(x16)
x17 = LayerNormalization()(x17)
c8 = Concatenate(axis=-1)([x17, x16])
x18 = TimeDistributed(Conv2DTranspose(64, (3, 3), strides =2, activation="gelu", padding="same", kernel_initializer='glorot_normal'))(c8)
x18 = LayerNormalization()(x18)
x19 = TimeDistributed(Conv2DTranspose(32, (3, 3), strides =2, activation="gelu", padding="same", kernel_initializer='glorot_normal'))(x18)
x19 = LayerNormalization()(x19)
    
TS = TimeDistributed(Conv2D(1, (1, 1), activation="linear", padding="same", kernel_initializer='glorot_normal'), name = 'TS_Final')(x19)    

Temporal_self_attention = Model(inputs = [input_img_ifgm, input_ts_ep], outputs = [TS, TS])
print(Temporal_self_attention.summary())
Temporal_self_attention.compile(optimizer=tensorflow.keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False), loss=['mean_squared_error', 'mean_squared_error'])

file_name_model_synth = 'Trained_models/Temporal_self_attention.hdf5' 
Temporal_self_attention.load_weights(file_name_model_synth)

# Test model for earthquake of Mw 5.7

In [ ]:
# Real earthquake 5.7
import tensorflow as tf

lines_test = [line.rstrip('\\n') for line in open('EQ_data_20_03_2019_text_file.txt')] #read text file that contains location and names of data
data_folder_name = 'Turkey_EQ_data_20_03_2019'
all_indices = list(np.arange(len(lines_test)))
grp_seq_each_v_patch = 182  # Total data: 7 Interferrogram data sequences of size 26x256x256 = 7 x 26 =128 (change as per test data)
Total_synth_v_locations = 1 # Total patch locations = 1 (change this variable as per test data)
sh_indices_ts = all_indices[0 : ]
iterations_ts = np.int((Total_synth_v_locations) * grp_seq_each_v_patch/N_IFG) # number of iterations = number of data sequences

output_path = ('EQ_5.7_Output/')  # name of output folder

    
cc = 0 # counter set for patch locations greater than 1 
for i in range(0,Total_synth_v_locations):
    for ii in range(0,np.int(grp_seq_each_v_patch/N_IFG)):
            
        count_value = np.int(ii+cc)
       
        startIfg = count_value*N_IFG
        endIfg = startIfg + N_IFG
        
      
        test_image_path = lines_test[sh_indices_ts[startIfg]]   
        test_image_patchname = data_folder_name

        test_set = generator(sh_indices_ts,lines_test, count_value)
        print(test_image_patchname, 'Sequence Number: ', ii)
                 
        if ii == 0:
            ts_5 = ALADDIn_intermediate_model.predict(test_set)
            ts_5 = ts_5[:,0:5,:,:,:]
            TS_h, TS_f = Temporal_self_attention.predict([test_set, ts_5],batch_size=1) 
            IFG_pred = create_IFG(TS_f)
                
        if ii > 0:
            test_set_p = generator(sh_indices_ts,lines_test, count_value-1)
            ts_5p_h, ts_5p_f = Temporal_self_attention.predict([test_set_p, ts_5])
            ts_5 = ts_5p_f[:,4:,:,:,:]
            TS_h, TS_f = Temporal_self_attention.predict([test_set, ts_5],batch_size=1) 
            IFG_pred = create_IFG(TS_f)
           
        
        sub_folder_path = output_path + test_image_patchname + '/' + str(ii) + '_ifg/'
        if not os.path.exists(sub_folder_path):
            os.makedirs(sub_folder_path)

        save_IFG(startIfg, endIfg, lines_test, sh_indices_ts, test_set, IFG_pred, sub_folder_path) ## save all input and output for comparison (GT = ground truth or input interferograms, REC = reconstructed interferograms, RES = residual interferograms (GT- REC)) )
            
        
        sub_folder_path1 = output_path + test_image_patchname + '/' + str(ii) + '_TS/'
        if not os.path.exists(sub_folder_path1):
            os.makedirs(sub_folder_path1)
           
        TS_names = get_names_TS(startIfg, endIfg, lines_test, sh_indices_ts)
        
        save_EPOCHS(TS_names, TS_f, sub_folder_path1) ## save output epoch time-series
            
                    
    #cc = cc + iterations_ts  ## uncomment and increment this counter if testing for greater than 1 patch locations
        

# Test model for earthquake with shuttling

In [ ]:
# shutteling F1 F2 F3 B3 B2 B1 real eq
import tensorflow as tf

shuttling_itr = 2 # number of times you want to shuttle
lines_test = [line.rstrip('\\n') for line in open('EQ_data_20_03_2019_text_file.txt')] # open text file and read data
data_folder_name = 'Turkey_EQ_data_20_03_2019'
all_indices = list(np.arange(len(lines_test)))
grp_seq_each_v_patch = 182 # Total data: 7 Interferrogram data sequences of size 26x256x256 = 7 x 26 =128 (change as per test data)
Total_synth_v_locations = 1 # Total patch locations = 1 (change this variable as per test data) 
sh_indices_ts = all_indices[0 : ]
iterations_ts = np.int((Total_synth_v_locations) * grp_seq_each_v_patch/N_IFG) #number of iterations = number of data sequences

output_path = ('EQ_5.7_Output_SHUTTLING/') # name of output folder

cc = 0
    
for i in range(0,Total_synth_v_locations):
    for shtl in range(0,shuttling_itr):
        for ii in range(0,np.int(grp_seq_each_v_patch/N_IFG)):
                    
            count_value = np.int(ii+cc)
                    
            startIfg = count_value*N_IFG
            endIfg = startIfg + N_IFG
                 
            test_image_path = lines_test[sh_indices_ts[startIfg]]    
            test_image_patchname = data_folder_name
            
            print(test_image_patchname,'Forward Shuttle number: ', shtl, 'Sequnce number: ', ii)

                 
            test_set = generator(sh_indices_ts,lines_test, count_value)
                    
            if shtl == 0:
                if ii == 0:
                    ts_5 = ALADDIn_intermediate_model.predict(test_set)
                    ts_5 = ts_5[:,0:5,:,:,:]
                    TS_h, TS_f = Temporal_self_attention.predict([test_set, ts_5],batch_size=1) 
                    IFG_pred = create_IFG(TS_f)
    
                     
                if ii > 0:
                    ts_5 = TS_f[:,4:,:,:,:]
                    TS_h, TS_f = Temporal_self_attention.predict([test_set, ts_5],batch_size=1) 
                    IFG_pred = create_IFG(TS_f)
 
                
            if shtl > 0:
                if ii == 0:
                    ts_5[:,4,:,:,:] = TS_f[:,4,:,:,:]
                    ts_5[:,3,:,:,:] = TS_f[:,5,:,:,:]
                    ts_5[:,2,:,:,:] = TS_f[:,6,:,:,:]
                    ts_5[:,1,:,:,:] = TS_f[:,7,:,:,:]
                    ts_5[:,0,:,:,:] = TS_f[:,8,:,:,:]
                    TS_h, TS_f = Temporal_self_attention.predict([test_set, ts_5],batch_size=1) 
                    IFG_pred = create_IFG(TS_f)
                    #print(test_set.shape)
                    #print(IFG_pred.shape) 
                            
                if ii > 0:
                    ts_5 = TS_f[:,4:,:,:,:]
                    TS_h, TS_f = Temporal_self_attention.predict([test_set, ts_5],batch_size=1) 
                    IFG_pred = create_IFG(TS_f) 
                            
                            
            sub_folder_path = output_path + test_image_patchname + '/F_shtl_' + str(shtl) + '/' + str(ii) + '_ifg/'
                 
            if not os.path.exists(sub_folder_path):
                os.makedirs(sub_folder_path)
                    
                 
            save_IFG(startIfg, endIfg, lines_test, sh_indices_ts, test_set, IFG_pred, sub_folder_path)

                 
            sub_folder_path1 = output_path + test_image_patchname + '/F_shtl_' + str(shtl) + '/' + str(ii) + '_TS/'
            if not os.path.exists(sub_folder_path1):
                os.makedirs(sub_folder_path1)
                    
               
            TS_names = get_names_TS(startIfg, endIfg, lines_test, sh_indices_ts)
        
            save_EPOCHS(TS_names, TS_f, sub_folder_path1)
                
                        

        for ii in range(0, np.int(grp_seq_each_v_patch/N_IFG)):
            b_count_value =  count_value - ii
                   
            startIfg = b_count_value*N_IFG + N_IFG-1
            endIfg = startIfg - N_IFG
                    
            bkwd_names_in = bkwd_names_indx(startIfg, endIfg)
                 
            test_image_path = lines_test[sh_indices_ts[startIfg]]    
            test_image_patchname = data_folder_name
                 
            test_set = generator_bkwd(sh_indices_ts,lines_test, b_count_value)
            print(test_image_patchname, 'Backward Shuttle number: ', shtl, 'Sequnce number: ', ii)
                    
            if ii == 0:
                ts_5[:,0,:,:,:] = TS_f[:,8,:,:,:]
                ts_5[:,1,:,:,:] = TS_f[:,7,:,:,:]
                ts_5[:,2,:,:,:] = TS_f[:,6,:,:,:]
                ts_5[:,3,:,:,:] = TS_f[:,5,:,:,:]
                ts_5[:,4,:,:,:] = TS_f[:,4,:,:,:]
                TS_h, TS_f = Temporal_self_attention.predict([test_set, ts_5],batch_size=1) 
                IFG_pred = create_IFG(TS_f)
                 
                                           
            if ii > 0:
                ts_5 = TS_f[:,4:,:,:,:]
                TS_h, TS_f = Temporal_self_attention.predict([test_set, ts_5],batch_size=1) 
                IFG_pred = create_IFG(TS_f) 
                        
            sub_folder_path = output_path + test_image_patchname + '/B_shtl_' + str(shtl) + '/' + str(ii) + '_ifg/'
                 
            if not os.path.exists(sub_folder_path):
                os.makedirs(sub_folder_path)
                    
                    
            save_IFG_backward(startIfg, endIfg, lines_test, sh_indices_ts, bkwd_names_in, test_set, IFG_pred, sub_folder_path)
                    
            sub_folder_path1 = output_path + test_image_patchname + '/B_shtl_' + str(shtl) + '/' + str(ii) + '_TS/'
                    
            if not os.path.exists(sub_folder_path1):
                os.makedirs(sub_folder_path1)
                         
            TS_names = get_names_TS_backwards(startIfg, endIfg, lines_test, sh_indices_ts)
            save_EPOCHS_BACKWARDS(TS_names, TS_f, sub_folder_path1)       
             
    #cc = cc + iterations_ts  ## uncomment and increment this counter if testing for greater than 1 patch locations